In [68]:
# Imports
import pandas as pd
import numpy as np
import io
import pytz
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from pandas.tseries.offsets import DateOffset
import yfinance as yf
import pandas_ta as ta

## Step 1: Create the Stochastic Trading Algorithm


In [69]:
#import the S&P 500 data

df = pd.read_csv(
    Path("../Resources/spy.csv"),
    index_col = 'Date',
    infer_datetime_format=True,
    parse_dates=True
)

df.head()

,Close,Open,High,Volume,Low
Date,,,,,
2021-04-13 09:30:00-04:00,411.950012,411.529999,412.149994,11677340,411.119995
2021-04-13 10:30:00-04:00,412.059998,411.950012,412.200012,5872073,411.720001
2021-04-13 11:30:00-04:00,411.959991,412.059906,412.209991,3257155,411.739990
2021-04-13 12:30:00-04:00,411.970001,411.964996,412.119995,6086220,411.540009
2021-04-13 13:30:00-04:00,412.698090,411.975006,412.720001,7161962,411.940002


In [70]:
# make a copy of the DataFrame
stoch_df = df.copy()

stoch_df.head()

,Close,Open,High,Volume,Low
Date,,,,,
2021-04-13 09:30:00-04:00,411.950012,411.529999,412.149994,11677340,411.119995
2021-04-13 10:30:00-04:00,412.059998,411.950012,412.200012,5872073,411.720001
2021-04-13 11:30:00-04:00,411.959991,412.059906,412.209991,3257155,411.739990
2021-04-13 12:30:00-04:00,411.970001,411.964996,412.119995,6086220,411.540009
2021-04-13 13:30:00-04:00,412.698090,411.975006,412.720001,7161962,411.940002


In [71]:
# Calculate stochastic values using the pandas_ta library
stoch_df.ta.stoch(close="Close", append=True)

,STOCHk_14_3_3,STOCHd_14_3_3
Date,,
2021-04-14 15:30:00-04:00,NaN,NaN
2021-04-15 09:30:00-04:00,NaN,NaN
2021-04-15 10:30:00-04:00,67.333624,NaN
2021-04-15 11:30:00-04:00,91.980541,NaN
2021-04-15 12:30:00-04:00,89.150989,82.821718
...,...,...
2022-04-12 12:30:00-04:00,28.591089,30.039237
2022-04-12 13:30:00-04:00,14.954800,25.692469
2022-04-12 14:30:00-04:00,8.822045,17.455978


In [72]:
#drop NaN values

stoch_df = stoch_df.dropna()

stoch_df.head()

,Close,Open,High,Volume,Low,STOCHk_14_3_3,STOCHd_14_3_3
Date,,,,,,,
2021-04-15 12:30:00-04:00,415.290009,415.179993,415.700012,5138590,415.089996,89.150989,82.821718
2021-04-15 13:30:00-04:00,415.609985,415.299011,415.670105,5492613,415.239990,94.443276,91.858269
2021-04-15 14:30:00-04:00,414.979095,415.609985,415.790009,7493486,414.890015,91.055170,91.549812
2021-04-15 15:30:00-04:00,415.829987,414.970001,416.160004,12386540,414.950012,91.805223,92.434556
2021-04-16 09:30:00-04:00,416.420013,417.250000,417.390015,18984659,415.730011,87.467433,90.109275


In [73]:
# set the share size to 100
share_size = 100

In [74]:
stoch_df["k-d"] = stoch_df["STOCHk_14_3_3"] - stoch_df["STOCHd_14_3_3"]

C:\Users\nmend\AppData\Local\Temp/ipykernel_22236/2976170539.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stoch_df["k-d"] = stoch_df["STOCHk_14_3_3"] - stoch_df["STOCHd_14_3_3"]


In [75]:
#set the initial_capital to 100000

initial_capital = 100000

In [76]:
#Create the Stochastic signal

stoch_df["Signal"]=0.0

stoch_df["Signal"] = np.where(
    stoch_df["k-d"] > 0, 1.0, 0.0
)

#calculate entry/exit points, 1 or -1
stoch_df["Entry/Exit"] = stoch_df["Signal"].diff()

#review the dataframe
stoch_df.tail(10)

C:\Users\nmend\AppData\Local\Temp/ipykernel_22236/2661355121.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stoch_df["Signal"]=0.0
C:\Users\nmend\AppData\Local\Temp/ipykernel_22236/2661355121.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stoch_df["Signal"] = np.where(
C:\Users\nmend\AppData\Local\Temp/ipykernel_22236/2661355121.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

,Close,Open,High,Volume,Low,STOCHk_14_3_3,STOCHd_14_3_3,k-d,Signal,Entry/Exit
Date,,,,,,,,,,
2022-04-11 14:30:00-04:00,440.959991,441.529999,443.000000,10120550,440.859985,2.530476,2.839252,-0.308776,0.0,0.0
2022-04-11 15:30:00-04:00,439.959991,440.950012,441.019989,18938227,439.390015,2.864608,2.693896,0.170712,1.0,1.0
2022-04-12 09:30:00-04:00,443.589996,443.079987,445.750000,18679011,442.369995,14.484915,6.626667,7.858248,1.0,0.0
2022-04-12 10:30:00-04:00,444.059998,443.600006,444.230011,9075047,442.219910,27.995106,15.114876,12.880229,1.0,0.0
2022-04-12 11:30:00-04:00,441.790009,444.070007,444.500000,6698770,441.630005,33.531517,25.337179,8.194338,1.0,0.0
2022-04-12 12:30:00-04:00,441.640015,441.750000,442.700012,7124676,441.170013,28.591089,30.039237,-1.448148,0.0,-1.0
2022-04-12 13:30:00-04:00,438.489990,441.649994,441.940002,10061518,438.420013,14.954800,25.692469,-10.737669,0.0,0.0
2022-04-12 14:30:00-04:00,437.059998,438.480011,439.290009,14347270,436.650085,8.822045,17.455978,-8.633933,0.0,0.0
2022-04-12 15:30:00-04:00,438.269989,437.059906,438.779999,12496573,436.679993,7.240768,10.339204,-3.098436,0.0,0.0


In [77]:
#create a column named "Position" by multiplying the share_size by the signal
# buy a position when the Stochastic signal = 1 (k-d is positive)
# sell a position when the Stochastic signal = 0 (k-d is negative)

stoch_df["Position"] = share_size * stoch_df["Signal"]

#review the dataframe
stoch_df.head()

C:\Users\nmend\AppData\Local\Temp/ipykernel_22236/2517851076.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stoch_df["Position"] = share_size * stoch_df["Signal"]


,Close,Open,High,Volume,Low,STOCHk_14_3_3,STOCHd_14_3_3,k-d,Signal,Entry/Exit,Position
Date,,,,,,,,,,,
2021-04-15 12:30:00-04:00,415.290009,415.179993,415.700012,5138590,415.089996,89.150989,82.821718,6.329271,1.0,NaN,100.0
2021-04-15 13:30:00-04:00,415.609985,415.299011,415.670105,5492613,415.239990,94.443276,91.858269,2.585008,1.0,0.0,100.0
2021-04-15 14:30:00-04:00,414.979095,415.609985,415.790009,7493486,414.890015,91.055170,91.549812,-0.494642,0.0,-1.0,0.0
2021-04-15 15:30:00-04:00,415.829987,414.970001,416.160004,12386540,414.950012,91.805223,92.434556,-0.629333,0.0,0.0,0.0
2021-04-16 09:30:00-04:00,416.420013,417.250000,417.390015,18984659,415.730011,87.467433,90.109275,-2.641842,0.0,0.0,0.0


In [78]:
#find the points in time where a position is purchased or sold
stoch_df["Entry/Exit Position"] = stoch_df["Position"].diff()

#review the dataframe
stoch_df.head()


C:\Users\nmend\AppData\Local\Temp/ipykernel_22236/3533554596.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stoch_df["Entry/Exit Position"] = stoch_df["Position"].diff()


,Close,Open,High,Volume,Low,STOCHk_14_3_3,STOCHd_14_3_3,k-d,Signal,Entry/Exit,Position,Entry/Exit Position
Date,,,,,,,,,,,,
2021-04-15 12:30:00-04:00,415.290009,415.179993,415.700012,5138590,415.089996,89.150989,82.821718,6.329271,1.0,NaN,100.0,NaN
2021-04-15 13:30:00-04:00,415.609985,415.299011,415.670105,5492613,415.239990,94.443276,91.858269,2.585008,1.0,0.0,100.0,0.0
2021-04-15 14:30:00-04:00,414.979095,415.609985,415.790009,7493486,414.890015,91.055170,91.549812,-0.494642,0.0,-1.0,0.0,-100.0
2021-04-15 15:30:00-04:00,415.829987,414.970001,416.160004,12386540,414.950012,91.805223,92.434556,-0.629333,0.0,0.0,0.0,0.0
2021-04-16 09:30:00-04:00,416.420013,417.250000,417.390015,18984659,415.730011,87.467433,90.109275,-2.641842,0.0,0.0,0.0,0.0


In [79]:
#drop NaN values
stoch_df = stoch_df.dropna()

In [80]:
#create a Portfolio Holdings column by multiplying the Close price by the Position

stoch_df["Portfolio Holdings"] = stoch_df["Close"] * stoch_df["Position"]

#skip to the second trading day of the period to avoid issues surrounding a signal in hour 2
stoch_df = stoch_df.iloc[6: , :]

#review the dataframe
stoch_df.head()

,Close,Open,High,Volume,Low,STOCHk_14_3_3,STOCHd_14_3_3,k-d,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings
Date,,,,,,,,,,,,,
2021-04-16 12:30:00-04:00,416.549988,416.750000,416.869995,4907994,416.429993,86.937392,86.891916,0.045476,1.0,1.0,100.0,100.0,41654.998779
2021-04-16 13:30:00-04:00,417.070892,416.559998,417.170013,6359601,416.399994,90.801849,88.004048,2.797800,1.0,0.0,100.0,0.0,41707.089233
2021-04-16 14:30:00-04:00,417.500397,417.075012,417.589996,8440599,417.070007,93.629455,90.456232,3.173223,1.0,0.0,100.0,0.0,41750.039673
2021-04-16 15:30:00-04:00,417.299988,417.505005,417.910004,16933002,417.100006,93.105608,92.512304,0.593304,1.0,0.0,100.0,0.0,41729.998779
2021-04-19 09:30:00-04:00,415.785004,416.399994,416.739990,14033882,415.070007,73.842410,86.859158,-13.016748,0.0,-1.0,0.0,-100.0,0.000000


In [81]:
#To calculate Portfolio Cash, subtrace the cumulative sum of the trade cost/proceeds from the initial_capital
#The trade cost proceeds are calculated by multiplying the Close price by the Entry/Exit Position

stoch_df["Portfolio Cash"] = initial_capital - (stoch_df["Close"] * stoch_df["Entry/Exit Position"]).cumsum()

#review the dataframe
stoch_df.tail(10)

,Close,Open,High,Volume,Low,STOCHk_14_3_3,STOCHd_14_3_3,k-d,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash
Date,,,,,,,,,,,,,,
2022-04-11 14:30:00-04:00,440.959991,441.529999,443.000000,10120550,440.859985,2.530476,2.839252,-0.308776,0.0,0.0,0.0,0.0,0.000000,106212.670898
2022-04-11 15:30:00-04:00,439.959991,440.950012,441.019989,18938227,439.390015,2.864608,2.693896,0.170712,1.0,1.0,100.0,100.0,43995.999146,62216.671753
2022-04-12 09:30:00-04:00,443.589996,443.079987,445.750000,18679011,442.369995,14.484915,6.626667,7.858248,1.0,0.0,100.0,0.0,44358.999634,62216.671753
2022-04-12 10:30:00-04:00,444.059998,443.600006,444.230011,9075047,442.219910,27.995106,15.114876,12.880229,1.0,0.0,100.0,0.0,44405.999756,62216.671753
2022-04-12 11:30:00-04:00,441.790009,444.070007,444.500000,6698770,441.630005,33.531517,25.337179,8.194338,1.0,0.0,100.0,0.0,44179.000854,62216.671753
2022-04-12 12:30:00-04:00,441.640015,441.750000,442.700012,7124676,441.170013,28.591089,30.039237,-1.448148,0.0,-1.0,0.0,-100.0,0.000000,106380.673218
2022-04-12 13:30:00-04:00,438.489990,441.649994,441.940002,10061518,438.420013,14.954800,25.692469,-10.737669,0.0,0.0,0.0,0.0,0.000000,106380.673218
2022-04-12 14:30:00-04:00,437.059998,438.480011,439.290009,14347270,436.650085,8.822045,17.455978,-8.633933,0.0,0.0,0.0,0.0,0.000000,106380.673218
2022-04-12 15:30:00-04:00,438.269989,437.059906,438.779999,12496573,436.679993,7.240768,10.339204,-3.098436,0.0,0.0,0.0,0.0,0.000000,106380.673218


In [82]:
#calculate the Portfolo Total by adding Portfolio Cash and Portfolio Holdings
stoch_df["Portfolio Total"] = stoch_df["Portfolio Cash"] + stoch_df["Portfolio Holdings"]

#review the dataframe
stoch_df.tail()

,Close,Open,High,Volume,Low,STOCHk_14_3_3,STOCHd_14_3_3,k-d,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total
Date,,,,,,,,,,,,,,,
2022-04-12 12:30:00-04:00,441.640015,441.750000,442.700012,7124676,441.170013,28.591089,30.039237,-1.448148,0.0,-1.0,0.0,-100.0,0.000000,106380.673218,106380.673218
2022-04-12 13:30:00-04:00,438.489990,441.649994,441.940002,10061518,438.420013,14.954800,25.692469,-10.737669,0.0,0.0,0.0,0.0,0.000000,106380.673218,106380.673218
2022-04-12 14:30:00-04:00,437.059998,438.480011,439.290009,14347270,436.650085,8.822045,17.455978,-8.633933,0.0,0.0,0.0,0.0,0.000000,106380.673218,106380.673218
2022-04-12 15:30:00-04:00,438.269989,437.059906,438.779999,12496573,436.679993,7.240768,10.339204,-3.098436,0.0,0.0,0.0,0.0,0.000000,106380.673218,106380.673218
2022-04-12 16:00:00-04:00,438.290009,438.290009,438.290009,0,438.290009,13.034849,9.699221,3.335629,1.0,1.0,100.0,100.0,43829.000854,62551.672363,106380.673218


In [83]:
#calculate the Portfolio Period Returns based on the Portfolio Total
stoch_df["Portfolio Period Returns"] = stoch_df["Portfolio Total"].pct_change()

#review the dataframe
stoch_df.tail()

,Close,Open,High,Volume,Low,STOCHk_14_3_3,STOCHd_14_3_3,k-d,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Period Returns
Date,,,,,,,,,,,,,,,,
2022-04-12 12:30:00-04:00,441.640015,441.750000,442.700012,7124676,441.170013,28.591089,30.039237,-1.448148,0.0,-1.0,0.0,-100.0,0.000000,106380.673218,106380.673218,-0.000141
2022-04-12 13:30:00-04:00,438.489990,441.649994,441.940002,10061518,438.420013,14.954800,25.692469,-10.737669,0.0,0.0,0.0,0.0,0.000000,106380.673218,106380.673218,0.000000
2022-04-12 14:30:00-04:00,437.059998,438.480011,439.290009,14347270,436.650085,8.822045,17.455978,-8.633933,0.0,0.0,0.0,0.0,0.000000,106380.673218,106380.673218,0.000000
2022-04-12 15:30:00-04:00,438.269989,437.059906,438.779999,12496573,436.679993,7.240768,10.339204,-3.098436,0.0,0.0,0.0,0.0,0.000000,106380.673218,106380.673218,0.000000
2022-04-12 16:00:00-04:00,438.290009,438.290009,438.290009,0,438.290009,13.034849,9.699221,3.335629,1.0,1.0,100.0,100.0,43829.000854,62551.672363,106380.673218,0.000000


In [84]:
#Calculate the Portfolio Cumulative Returns based on the Portfolio Daily Returns
stoch_df["Portfolio Cumulative Returns"] = (1 + stoch_df["Portfolio Period Returns"]).cumprod() - 1

#review the dataframe
stoch_df.head(25)

,Close,Open,High,Volume,Low,STOCHk_14_3_3,STOCHd_14_3_3,k-d,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Period Returns,Portfolio Cumulative Returns
Date,,,,,,,,,,,,,,,,,
2021-04-16 12:30:00-04:00,416.549988,416.750000,416.869995,4907994,416.429993,86.937392,86.891916,0.045476,1.0,1.0,100.0,100.0,41654.998779,58345.001221,100000.000000,NaN,NaN
2021-04-16 13:30:00-04:00,417.070892,416.559998,417.170013,6359601,416.399994,90.801849,88.004048,2.797800,1.0,0.0,100.0,0.0,41707.089233,58345.001221,100052.090454,0.000521,0.000521
2021-04-16 14:30:00-04:00,417.500397,417.075012,417.589996,8440599,417.070007,93.629455,90.456232,3.173223,1.0,0.0,100.0,0.0,41750.039673,58345.001221,100095.040894,0.000429,0.000950
2021-04-16 15:30:00-04:00,417.299988,417.505005,417.910004,16933002,417.100006,93.105608,92.512304,0.593304,1.0,0.0,100.0,0.0,41729.998779,58345.001221,100075.000000,-0.000200,0.000750
2021-04-19 09:30:00-04:00,415.785004,416.399994,416.739990,14033882,415.070007,73.842410,86.859158,-13.016748,0.0,-1.0,0.0,-100.0,0.000000,99923.501587,99923.501587,-0.001514,-0.000765
2021-04-19 10:30:00-04:00,415.640015,415.790009,416.065002,10137648,414.380005,52.851632,73.266550,-20.414918,0.0,0.0,0.0,0.0,0.000000,99923.501587,99923.501587,0.000000,-0.000765
2021-04-19 11:30:00-04:00,414.529999,415.640015,415.670013,10090917,414.230011,27.054034,51.249359,-24.195324,0.0,0.0,0.0,0.0,0.000000,99923.501587,99923.501587,0.000000,-0.000765
2021-04-19 12:30:00-04:00,414.630005,414.540009,414.940002,7098715,413.790009,21.411494,33.772387,-12.360893,0.0,0.0,0.0,0.0,0.000000,99923.501587,99923.501587,0.000000,-0.000765
2021-04-19 13:30:00-04:00,415.010010,414.609985,415.089996,4790001,414.380005,19.383953,22.616494,-3.232541,0.0,0.0,0.0,0.0,0.000000,99923.501587,99923.501587,0.000000,-0.000765


## Visualize the Stochastic Algorithm

In [85]:
#visualize entry positions relative to close price

entry = stoch_df[stoch_df["Entry/Exit"] == 1.0]["Close"].hvplot.scatter(
    color = 'purple',
    marker = '^',
    legend = False,
    ylabel = "Price in $",
    width = 1400,
    height = 600)

#visualize the exit positions relative to close price
exit = stoch_df[stoch_df["Entry/Exit"] == -1.0]["Close"].hvplot.scatter(
    color = 'orange',
    marker = 'v',
    legend = False,
    ylabel = "Price in $",
    width = 1400,
    height = 600)

#visualize the close price for investment
security_close = stoch_df[['Close']].hvplot(
    line_color = "lightblue",
    ylabel = "Price in $",
    width = 1400,
    height = 600)

entry_exit_plot = security_close * entry * exit
entry_exit_plot.opts(title = "Stochastic Trading Algorithm Entry/Exits")

entry_exit_plot


:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)

In [86]:
#visualize entry positions relative to close price

entry = stoch_df[stoch_df["Entry/Exit"] == 1.0]["Portfolio Total"].hvplot.scatter(
    color = 'purple',
    marker = '^',
    legend = False,
    ylabel = "Total Portfolio Value",
    width = 1400,
    height = 600)

#visualize the exit positions relative to close price
exit = stoch_df[stoch_df["Entry/Exit"] == -1.0]["Portfolio Total"].hvplot.scatter(
    color = 'orange',
    marker = 'v',
    legend = False,
    ylabel = "Total Portfolio Value",
    width = 1400,
    height = 600)

#visualize the Portfolio Total for investment
total_portfolio_value = stoch_df[['Portfolio Total']].hvplot(
    line_color = "lightblue",
    ylabel = "Total Portfolio Value",
    width = 1400,
    height = 600)

portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(
    title = "Stochastic Trading Algorithm - Total Portfolio Value",
    yformatter="%.0f"
)

portfolio_entry_exit_plot

:Overlay
   .Curve.I                    :Curve   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [Date]   (Portfolio Total)

## Evaluate the portfolio metrics of the Stochastic Algorithm

In [87]:
#create a new DataFrame to evaluate the performance of the algorithm

# create a list for the column name

columns = ["Backtest"]

metrics = [

    "Annualized Return",
    "Cumulative Returns",
    "Annual Volatility",
    "Sharpe Ratio"
    ]
portfolio_evaluation_df = pd.DataFrame(index = metrics, columns=columns)

portfolio_evaluation_df.head()

,Backtest
Annualized Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN


In [88]:
# calculate the annualized return

portfolio_evaluation_df.loc["Annualized Return"] = (

    stoch_df["Portfolio Period Returns"].mean() * 252 * 7

)

portfolio_evaluation_df

,Backtest
Annualized Return,0.063369
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN


In [89]:
# calculate the cumulative return

portfolio_evaluation_df.loc["Cumulative Returns"] = (

    stoch_df["Portfolio Cumulative Returns"][-1]
)

portfolio_evaluation_df

,Backtest
Annualized Return,0.063369
Cumulative Returns,0.063807
Annual Volatility,NaN
Sharpe Ratio,NaN


In [90]:
# Calculate the annual volatility

portfolio_evaluation_df.loc["Annual Volatility"] = (

    stoch_df["Portfolio Period Returns"].std() * np.sqrt(252*7)

)

portfolio_evaluation_df

,Backtest
Annualized Return,0.063369
Cumulative Returns,0.063807
Annual Volatility,0.045936
Sharpe Ratio,NaN


In [91]:
# Calculate the Sharpe Ratio

portfolio_evaluation_df.loc["Sharpe Ratio"] = (

    stoch_df["Portfolio Period Returns"].mean() * 252) / (

    stoch_df["Portfolio Period Returns"].std() * np.sqrt(252)

)

portfolio_evaluation_df

,Backtest
Annualized Return,0.063369
Cumulative Returns,0.063807
Annual Volatility,0.045936
Sharpe Ratio,0.521399


## Trade-Level Analytics

In [92]:
stoch_df["value"] =  stoch_df["Entry/Exit Position"] * stoch_df["Close"]

In [98]:
stoch_df["Entry/Exit Position"].value_counts()

 0.0      1335
 100.0     209
-100.0     208
Name: Entry/Exit Position, dtype: int64

In [93]:
# create a new dataframe to evaluate trade-level Risk/Reward Metrics

trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock',
        'Entry Date',
        'Exit Date',
        'Shares',
        'Entry Share Price',
        'Exit Share Price',
        'Entry Portfolio Value',
        'Exit Portfolio Value',
        'Profit/Loss']
)


In [94]:
#initialize the iterative values
entry_date = ""
exit_date = ""
entry_portfolio_value = 0.0
exit_portfolio_value = 0.0
share_size = 0.0
entry_share_price = 0.0
exit_share_price = 0.0


In [95]:
# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit
# Then append the record to the trade evaluation DataFrame
for index, row in stoch_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_value = row['value']
        share_size = row['Entry/Exit Position']
        entry_share_price = row['Close']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_value = abs(row['Close'] * row['Entry/Exit Position'])
        exit_share_price = row['Close']
        profit_loss = exit_portfolio_value - entry_portfolio_value
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'SPY',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Value': entry_portfolio_value,
                'Exit Portfolio Value': exit_portfolio_value,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Value,Exit Portfolio Value,Profit/Loss
0,SPY,2021-04-16 12:30:00-04:00,2021-04-19 09:30:00-04:00,100.0,416.549988,415.785004,41654.998779,41578.500366,-76.498413
1,SPY,2021-04-19 14:30:00-04:00,2021-04-20 09:30:00-04:00,100.0,414.325012,413.920013,41432.501221,41392.001343,-40.499878
2,SPY,2021-04-20 13:30:00-04:00,2021-04-20 14:30:00-04:00,100.0,411.535004,411.904999,41153.500366,41190.499878,36.999512
3,SPY,2021-04-20 15:30:00-04:00,2021-04-22 09:30:00-04:00,100.0,412.239990,415.329987,41223.999023,41532.998657,308.999634
4,SPY,2021-04-22 11:30:00-04:00,2021-04-22 12:30:00-04:00,100.0,416.359985,414.700012,41635.998535,41470.001221,-165.997314
...,...,...,...,...,...,...,...,...,...
203,SPY,2022-03-29 14:30:00-04:00,2022-03-30 10:30:00-04:00,100.0,461.515015,459.160004,46151.501465,45916.000366,-235.501099
204,SPY,2022-04-01 10:30:00-04:00,2022-04-05 09:30:00-04:00,100.0,451.695007,454.440002,45169.500732,45444.000244,274.499512
205,SPY,2022-04-06 13:30:00-04:00,2022-04-07 10:30:00-04:00,100.0,444.565002,444.470001,44456.500244,44447.000122,-9.500122
206,SPY,2022-04-07 12:30:00-04:00,2022-04-08 13:30:00-04:00,100.0,446.549988,448.029999,44654.998779,44802.999878,148.001099


In [96]:
#determine a winning vs. losing trade

win = 0
loss = 0
max_win = trade_evaluation_df["Profit/Loss"].max()
max_loss = trade_evaluation_df["Profit/Loss"].min()

for pnl in trade_evaluation_df["Profit/Loss"]:
    if pnl > 0:
        win +=1
    else:
        loss +=1

winrate = win / (win + loss)

print(f"There were {win} winning trades and {loss} losing trades giving us a winrate of {winrate})")
print(f"The largest gain made was ${max_win} and the largest loss was ${max_loss}")

There were 88 winning trades and 120 losing trades giving us a winrate of 0.4230769230769231)
The largest gain made was $2161.001586914055 and the largest loss was $-883.5021972656177


In [97]:
#let's see how much money our system made in total:

total_profit_loss = trade_evaluation_df["Profit/Loss"].sum()

print(f"Using our algorithm made ${total_profit_loss}!")

Using our algorithm made $6380.673217773394!
